In [2]:
import pandas as pd
import numpy as np

In [3]:
CA_data = pd.read_csv('../CA_data.csv')

### Dropping useless column


In [4]:
# drop these columns
columns_dropped = ['Vehicle 1 Business Name', 'Lat1', 'Lat2', 'Lat3', 'Lng1', 'Lng2', 'Lng3', 
                   'Property Damage', 'Property Owner\'s Details', 'ZIPCODE of Accident', 
                   'State of Accident', 'County of Accident', 'Location of Accident',
                   ]
CA_data.drop(columns=columns_dropped, inplace=True)


In [5]:
print(len(CA_data.columns))

68


#### Column Weather Vehicle 1
We are grouping Fog/visiblity with Dark because they are visual impariements\
We are grouping slippery, wind with raining because when it rains, it usally reults in slipperiness and sometimes wind\

In [6]:
CA_data['Weather Vehicle 1'].replace(['Fog/Visibility', 'Dark'], 'Fog/Visibility/Dark', inplace=True)
CA_data['Weather Vehicle 1'].replace(['Slippery', 'Wind'], 'Raining', inplace=True)

print(CA_data['Weather Vehicle 1'].value_counts())

Weather Vehicle 1
Clear                  265
Cloudy                  29
Raining                 14
Not Available           13
Fog/Visibility/Dark      4
Name: count, dtype: int64


#### Column: Vehicle 1 lighting
Grouping no unusla condition and daylight b/c they basically the same\
Grouping Dusk/dawn with dark with street lights b/c they are relatively similar conditions


In [7]:
CA_data['Lighting Vehicle 1'].replace(['No unusual condition'], 'Daylight', inplace=True)
CA_data['Lighting Vehicle 1'].replace(['Dusk/Dawn'], 'Dark w Street-lights', inplace=True)


print(CA_data['Lighting Vehicle 1'].value_counts())

Lighting Vehicle 1
Daylight                                227
Dark w Street-lights                     81
Not Available                            13
No unusual conditions                     2
Dark w Non-functioning Street-lights      2
Name: count, dtype: int64


#### Column: Surface/Road condition 
Wet roads = slippery roads\
Dropped other and proceeding straight b/c they dont provide much value

In [8]:

CA_data['Roadway Surface Vehicle 1'].replace(['Slippery'], 'Wet', inplace=True)
CA_data.drop(CA_data[(CA_data['Roadway Surface Vehicle 1'] == 'Other') | (CA_data['Roadway Surface Vehicle 1'] == 'Proceeding Straight')].index, inplace=True)

print(CA_data['Roadway Surface Vehicle 1'].value_counts())

Roadway Surface Vehicle 1
Dry              286
Wet               22
Not Available     15
Name: count, dtype: int64


#### Column: Movement before the collision
grouping turns, high movements, parking, and dropping others

In [9]:
CA_data['Movement Preceding Collision Vehicle 1'].replace(['Making Right Turn', 'Making Left Turn', 'Making U turn', 'Making Right Turn, Slowing/Stopping'], 'Turning', inplace=True)
CA_data['Movement Preceding Collision Vehicle 1'].replace(['Changing Lanes', 'Entering Traffic', 'Entrering Traffic','Xing into opposing lane', 'Passing Other Vehicle'], 'Highway movement', inplace=True)
CA_data['Movement Preceding Collision Vehicle 1'].replace(['Slowing/Stopping', 'Stopped in Traffic', 'Parked', 'Stopped, Merging'], 'Stopped', inplace=True)
CA_data['Movement Preceding Collision Vehicle 1'].replace(['Parking Manuerver'], 'Parking', inplace=True)
CA_data.drop(CA_data[(CA_data['Movement Preceding Collision Vehicle 1'] == 'Other')].index, inplace=True)

print(CA_data['Movement Preceding Collision Vehicle 1'].value_counts())

Movement Preceding Collision Vehicle 1
Stopped                152
Proceeding Straight     91
Turning                 29
Not Available           13
Parking                 11
Highway movement        10
Backing                  8
Parking Manuever         3
Stoppped                 1
Name: count, dtype: int64


#### Column: Vehicle 1 was stopped/moving in traffic
Fixing some off the mislabeled data


In [10]:

CA_data['Vehicle 1 was Stopped in Traffic'].replace(['/Off'], 'Yes', inplace=True)
CA_data['Vehicle 1 was Moving'].replace(['/Off'], 'No', inplace=True)

CA_data['Vehicle 1 was Stopped in Traffic'].replace(['yes'], 'Yes', inplace=True)
CA_data['Vehicle 1 was Moving'].replace(['Moving'], 'Yes', inplace=True)

print(CA_data['Vehicle 1 was Stopped in Traffic'].value_counts())  
print(CA_data['Vehicle 1 was Moving'].value_counts())

Vehicle 1 was Stopped in Traffic
No     203
Yes    169
Name: count, dtype: int64
Vehicle 1 was Moving
Yes    192
No     180
Name: count, dtype: int64


#### Combining pedestrian, bicycle, and other entitys into one column called External_columns


In [11]:
print(CA_data['Pedestrian Involved in Accident (w V1)'].value_counts())
print(CA_data['Bicyclist involved in Accident (w V1)'].value_counts())
print(CA_data['Any other involved entity (w V1)'].value_counts())

Pedestrian Involved in Accident (w V1)
No     371
Yes      1
Name: count, dtype: int64
Bicyclist involved in Accident (w V1)
No     363
Yes      9
Name: count, dtype: int64
Any other involved entity (w V1)
No                  342
Vehicle              13
Car                   5
Truck                 3
Parked                2
electric scooter      1
Const. Equipment      1
Scooter               1
IRON ROD              1
small dog             1
Van                   1
car                   1
Name: count, dtype: int64


In [12]:
def determine_new_col(row):
    pedestrian = row['Pedestrian Involved in Accident (w V1)'] == 'Yes'
    bicyclist = row['Bicyclist involved in Accident (w V1)'] == 'Yes'
    other = row['Any other involved entity (w V1)'].lower() not in ['no','vehicle','car','truck','van']
    if (pedestrian or bicyclist or other):
        return 1
    
    return 0

# Apply the function to each row
CA_data['External_factors'] = CA_data.apply(determine_new_col, axis=1)

print(CA_data['External_factors'].value_counts())

External_factors
0    355
1     17
Name: count, dtype: int64


#### Column: Vehicle 1 Manufactuer

In [22]:
# rint(CA_data['Vehicle 1 Manufacturer'].value_counts())

# if waymo, cruise, google, apple, etc is in the name replace with the company name
def determine_vehicle_manufactuer(row):
    companies = ['waymo', 'cruise', 'google', 'apple', 'zoox']
    for company in companies:
        if company in row['Vehicle 1 Manufacturer'].lower():
            return company
    return "other"

CA_data['Vehicle 1'] = CA_data.apply(determine_vehicle_manufactuer, axis=1)
print(CA_data['Vehicle 1'].value_counts())
print(CA_data['Vehicle 1 Make'].value_counts())


Vehicle 1
cruise    158
waymo     104
other      38
zoox       38
google     24
apple      10
Name: count, dtype: int64
Vehicle 1 Make
Chevrolet        117
Chrysler          60
Jaguar            49
Cruise            38
Toyota            37
Lexus             28
Lincoln            8
Google             7
Nissan             6
Ford               5
Volvo              3
MERCEDES-BENZ      3
Hyundai            2
NISSAN             2
Mercedes-Benz      2
Cheverolet         2
S CLASS            1
Navya              1
Audi               1
Name: count, dtype: int64
